<h1 style='color:blue'><center>Recommendation System using Graph Neural Networks</center></h1>

---

<b style='color:DodgerBlue'><center><a href='https://www.linkedin.com/in/mugheesasif/'>Mughees Asif</a></center></b>
<b style='color:DodgerBlue'><center><a href='https://www.sems.qmul.ac.uk/staff/a.nanjangud'>Dr Angadh Nanjangud</a></center></b>
<i style='color:rgb(0, 122, 172)'><center><a href='http://www.eecs.qmul.ac.uk/'>School of Electronic Engineering and Computer Science</a></center></i>
<i style='color:rgb(0, 122, 172)'><center><a href='https://www.qmul.ac.uk/'>Queen Mary, University of London</a></center></i>

<img src='./images/hero.jpg' alt="introduction" width="500px" />

## Abstract

In recent years, recommendation systems have been developed with graph neural networks (GNNs) to expedite the aggregation process of macro (e.g., topological structure) and micro (e.g., node information) operations to enrich information filtering capabilities. Additionally, using neural networks for recommendation tasks on graphlike data structures has been proven to elicit meaningful representations of user-item relationships. However, the representation learning process is not linear as social relations augmented with item interactions, both need to be considered for optimal results. This research project aims to address this issue by proposing a novel framework that is capable of using multiple preferences for outlining recommendations/ratings of specific items to users. The framework is also split into three variations that are pitted against published academic recommendation systems. The training and testing of the models is done by using two real-world datasets that contain user-item and user-user information. 

<img src="./images/rating.png" alt="rating" style="width: 500px;"/>
<h5 align="center">Figure 1: Research framework displaying the two main components needed to make a rating prediction</h5>

<div class="alert alert-block alert-danger">
<b>Warning:</b> <li>The written content of this notebook mirrors the content in the <a href='/'>official PDF evaluation</a>.</li> <li>Please view this notebook <a href='/'>here</a> to see the complete open-source version.</li>
</div>

## Contents<a class="anchor" id="contents"></a>

---

**1** &nbsp;&nbsp;**[Import dependencies](#dw-dep)**<br>

**2** &nbsp;&nbsp;**[Download data](#data)**<br>

**3** &nbsp;&nbsp;**[Data processing](#preprocess-data)**<br>

**4** &nbsp;&nbsp;**[Graph Neural Network](#gnn)**<br>

**5** &nbsp;&nbsp;**[Training](#training)**<br>

**6** &nbsp;&nbsp;**[Testing](#testing)**<br>

**7** &nbsp;&nbsp;**[Results](#results)**<br>
<!-- &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;6.1&nbsp;&nbsp;*[Live](#live)*<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;6.2&nbsp;&nbsp;*[Recorded](#recorded)*<br>
 -->
**8** &nbsp;&nbsp;**[References](#references)**<br>

<div class="alert alert-block alert-info">
<b>Tip:</b> To return to the contents, press the 🔝 icon located in the title of each chapter.</div>

## 1&nbsp;&nbsp;Import dependencies <a class="anchor" id="dw-dep"></a> [🔝](#contents)

This section imports the necessary libraries needed to develop the GNN recommendation system.

In [ ]:
import torch
import torch.nn as nn
import random
import pickle
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

## 2&nbsp;&nbsp;Download data <a class="anchor" id="data"></a> [🔝](#contents)

This section downloads the Ciao and Epinions data sets from the official [GitHub](https://github.com/mughees-asif/) repository of this project. They can also be sourced from [here](https://www.cse.msu.edu/~tangjili/datasetcode/truststudy.htm).

In [1]:
!wget https://github.com/mughees-asif/graph-rec/raw/master/data.zip
!unzip /content/data.zip

--2022-06-23 07:02:25--  https://github.com/mughees-asif/graph-rec/raw/master/data.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mughees-asif/graph-rec/master/data.zip [following]
--2022-06-23 07:02:25--  https://raw.githubusercontent.com/mughees-asif/graph-rec/master/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25230096 (24M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  24.06M  --.-KB/s    in 0.1s    

2022-06-23 07:02:27 (192 MB/s) - ‘data.zip’ saved [25230096/25230096]

Archive:  /content/data.zip
  inflating: ciao/rating.mat      

## 3&nbsp;&nbsp;Data processing <a class="anchor" id="preprocess-data"></a> [🔝](#contents)

This section will highlight the loading of the data for neural network training.

In [3]:
class GraphDataset(Dataset):
    def __init__(self, data, u_items_list, u_user_list, u_users_items_list, i_users_list):
        self.data = data
        self.u_items_list = u_items_list
        self.u_users_list = u_user_list
        self.u_users_items_list = u_users_items_list
        self.i_users_list = i_users_list
    
    def __getitem__(self, index):
        uid = self.data[index][0]
        iid = self.data[index][1]
        label = self.data[index][2]
        u_items = self.u_items_list[uid]
        u_users = self.u_users_list[uid]
        u_users_items = self.u_users_items_list[uid]
        i_users = self.i_users_list[iid]

        return (uid, iid, label), u_items, u_users, u_users_items, i_users

    def __len__(self):
        return len(self.data)


In [4]:
truncate_len = 45

def collate_fn(batch_data):

    uids, iids, labels = [], [], []
    u_items, u_users, u_users_items, i_users = [], [], [], []
    u_items_len, u_users_len, i_users_len = [], [], []

    for data, u_items_u, u_users_u, u_users_items_u, i_users_i in batch_data:

        (uid, iid, label) = data
        uids.append(uid)
        iids.append(iid)
        labels.append(label)

        # user-items
        if len(u_items_u) <= truncate_len:
            u_items.append(u_items_u)
        else:
            u_items.append(random.sample(u_items_u, truncate_len))
        u_items_len.append(min(len(u_items_u), truncate_len))
        
        # user-users and user-users-items
        if len(u_users_u) <= truncate_len:
            u_users.append(u_users_u)
            u_u_items = [] 
            for uui in u_users_items_u:
                if len(uui) < truncate_len:
                    u_u_items.append(uui)
                else:
                    u_u_items.append(random.sample(uui, truncate_len))
            u_users_items.append(u_u_items)
        else:
            sample_index = random.sample(list(range(len(u_users_u))), truncate_len)
            u_users.append([u_users_u[si] for si in sample_index])

            u_users_items_u_tr = [u_users_items_u[si] for si in sample_index]
            u_u_items = [] 
            for uui in u_users_items_u_tr:
                if len(uui) < truncate_len:
                    u_u_items.append(uui)
                else:
                    u_u_items.append(random.sample(uui, truncate_len))
            u_users_items.append(u_u_items)

        u_users_len.append(min(len(u_users_u), truncate_len))	

        # item-users
        if len(i_users_i) <= truncate_len:
            i_users.append(i_users_i)
        else:
            i_users.append(random.sample(i_users_i, truncate_len))
        i_users_len.append(min(len(i_users_i), truncate_len))

    batch_size = len(batch_data)

    # padding
    u_items_maxlen = max(u_items_len)
    u_users_maxlen = max(u_users_len)
    i_users_maxlen = max(i_users_len)
    
    u_item_pad = torch.zeros([batch_size, u_items_maxlen, 2], dtype=torch.long)
    for i, ui in enumerate(u_items):
        u_item_pad[i, :len(ui), :] = torch.LongTensor(ui)
    
    u_user_pad = torch.zeros([batch_size, u_users_maxlen], dtype=torch.long)
    for i, uu in enumerate(u_users):
        u_user_pad[i, :len(uu)] = torch.LongTensor(uu)
    
    u_user_item_pad = torch.zeros([batch_size, u_users_maxlen, u_items_maxlen, 2], dtype=torch.long)
    for i, uu_items in enumerate(u_users_items):
        for j, ui in enumerate(uu_items):
            u_user_item_pad[i, j, :len(ui), :] = torch.LongTensor(ui)

    i_user_pad = torch.zeros([batch_size, i_users_maxlen, 2], dtype=torch.long)
    for i, iu in enumerate(i_users):
        i_user_pad[i, :len(iu), :] = torch.LongTensor(iu)

    uids = torch.LongTensor(uids)
    iids = torch.LongTensor(iids)
    labels = torch.FloatTensor(labels)

    return uids, iids, labels, u_item_pad, u_user_pad, u_user_item_pad, i_user_pad

## 4&nbsp;&nbsp;Graph Neural Network (GNN)<a class="anchor" id="gnn"></a> [🔝](#contents)

In [5]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, input_dim//2, bias=True),
            nn.ReLU(),
            nn.Linear(input_dim//2, output_dim, bias=True)
        )

    def forward(self, x):
        return self.mlp(x)

class Aggregator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Aggregator, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, output_dim, bias=True),
            nn.ReLU()
        )

    def forward(self, x):
        return self.mlp(x)


class UserModel(nn.Module):
    def __init__(self, emb_dim, user_emb, item_emb, rating_emb):
        super(UserModel, self).__init__()
        self.emb_dim = emb_dim
        self.user_emb = user_emb
        self.item_emb = item_emb
        self.rating_emb = rating_emb

        self.g_v = MLP(2*self.emb_dim, self.emb_dim)
        
        self.user_item_attn = MLP(2*self.emb_dim, 1)
        self.aggr_items = Aggregator(self.emb_dim, self.emb_dim)

        self.user_user_attn = MLP(2*self.emb_dim, 1)
        self.aggr_neighbors = Aggregator(self.emb_dim, self.emb_dim)

        self.mlp = nn.Sequential(
            nn.Linear(2*self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU()
        )

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.eps = 1e-10

    def forward(self, uids, u_item_pad, u_user_pad, u_user_item_pad):

        q_a = self.item_emb(u_item_pad[:,:,0])
        u_item_er = self.rating_emb(u_item_pad[:,:,1])
        x_ia = self.g_v(torch.cat([q_a, u_item_er], dim=2).view(-1, 2*self.emb_dim)).view(q_a.size())
        mask_u = torch.where(u_item_pad[:,:,0]>0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        p_i = mask_u.unsqueeze(2).expand_as(x_ia) * self.user_emb(uids).unsqueeze(1).expand_as(x_ia)
        alpha = self.user_item_attn(torch.cat([x_ia, p_i], dim=2).view(-1, 2*self.emb_dim)).view(mask_u.size())
        alpha = torch.exp(alpha)*mask_u
        alpha = alpha / (torch.sum(alpha, 1).unsqueeze(1).expand_as(alpha) + self.eps)
        h_iI = self.aggr_items(torch.sum(alpha.unsqueeze(2).expand_as(x_ia) * x_ia, 1))


        q_a_s = self.item_emb(u_user_item_pad[:,:,:,0])
        u_user_item_er = self.rating_emb(u_user_item_pad[:,:,:,1])
        x_ia_s = self.g_v(torch.cat([q_a_s, u_user_item_er], dim=2).view(-1, 2*self.emb_dim)).view(q_a_s.size())
        mask_s = torch.where(u_user_item_pad[:,:,:,0]>0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        p_i_s = mask_s.unsqueeze(3).expand_as(x_ia_s) * self.user_emb(u_user_pad).unsqueeze(2).expand_as(x_ia_s)
        alpha_s = self.user_item_attn(torch.cat([x_ia_s, p_i_s], dim=3).view(-1, 2*self.emb_dim)).view(mask_s.size())
        alpha_s = torch.exp(alpha_s)*mask_s
        alpha_s = alpha_s / (torch.sum(alpha_s, 2).unsqueeze(2).expand_as(alpha_s) + self.eps)
        h_oI_temp = torch.sum(alpha_s.unsqueeze(3).expand_as(x_ia_s) * x_ia_s, 2)
        h_oI = self.aggr_items(h_oI_temp.view(-1, self.emb_dim)).view(h_oI_temp.size())
        
        beta = self.user_user_attn(torch.cat([h_oI, self.user_emb(u_user_pad)], dim = 2).view(-1, 2 * self.emb_dim)).view(u_user_pad.size())
        mask_su = torch.where(u_user_pad > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        beta = torch.exp(beta) * mask_su
        beta = beta / (torch.sum(beta, 1).unsqueeze(1).expand_as(beta) + self.eps)
        h_iS = self.aggr_neighbors(torch.sum(beta.unsqueeze(2).expand_as(h_oI) * h_oI, 1))

        h_i = self.mlp(torch.cat([h_iI, h_iS], dim = 1))

        return h_i


class ItemModel(nn.Module):
    def __init__(self, emb_dim, user_emb, item_emb, rating_emb):
        super(ItemModel, self).__init__()
        self.emb_dim = emb_dim
        self.user_emb = user_emb
        self.item_emb = item_emb
        self.rating_emb = rating_emb

        self.g_u = MLP(2*self.emb_dim, self.emb_dim)

        self.item_users_attn = MLP(2*self.emb_dim, 1)
        self.aggr_users = Aggregator(self.emb_dim, self.emb_dim)

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.eps = 1e-10
    
    def forward(self, iids, i_user_pad):

        p_t = self.user_emb(i_user_pad[:,:,0])
        i_user_er = self.rating_emb(i_user_pad[:,:,1])
        mask_i = torch.where(i_user_pad[:,:,0] > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        f_jt = self.g_u(torch.cat([p_t, i_user_er], dim = 2).view(-1, 2 * self.emb_dim)).view(p_t.size())
        q_j = mask_i.unsqueeze(2).expand_as(f_jt) * self.item_emb(iids).unsqueeze(1).expand_as(f_jt)
        mu_jt = self.item_users_attn(torch.cat([f_jt, q_j], dim = 2).view(-1, 2 * self.emb_dim)).view(mask_i.size())
        mu_jt = torch.exp(mu_jt) * mask_i
        mu_jt = mu_jt / (torch.sum(mu_jt, 1).unsqueeze(1).expand_as(mu_jt) + self.eps)
        
        z_j = self.aggr_users(torch.sum(mu_jt.unsqueeze(2).expand_as(f_jt) * f_jt, 1))

        return z_j
        
    
class GraphRec(nn.Module):
    def __init__(self, n_users, n_items, n_ratings, emb_dim = 64):
        super(GraphRec, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.n_ratings = n_ratings
        self.emb_dim = emb_dim

        self.user_emb = nn.Embedding(self.n_users, self.emb_dim, padding_idx=0)
        self.item_emb = nn.Embedding(self.n_items, self.emb_dim, padding_idx=0)
        self.rating_emb = nn.Embedding(self.n_ratings, self.emb_dim, padding_idx=0)

        self.user_model = UserModel(self.emb_dim, self.user_emb, self.item_emb, self.rating_emb)
        self.item_model = ItemModel(self.emb_dim, self.user_emb, self.item_emb, self.rating_emb)

        self.mlp = nn.Sequential(
            nn.Linear(2*self.emb_dim, self.emb_dim, bias=True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, self.emb_dim, bias=True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, 1)
        )

    def forward(self, uids, iids, u_item_pad, u_user_pad, u_user_item_pad, i_user_pad):

        h_i = self.user_model(uids, u_item_pad, u_user_pad, u_user_item_pad)
        z_j = self.item_model(iids, i_user_pad)

        r_ij = self.mlp(torch.cat([h_i, z_j], dim=1))

        return r_ij

## 5&nbsp;&nbsp;Training <a class="anchor" id="training"></a> [🔝](#contents)

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device - ' + str(device))
batch_size = 128
embed_dim = 64
learning_rate = 0.001
n_epochs = 10

device - cuda


### Read dataset and preprocess it to form batches

In [35]:
with open('/content/dataset_epinions.pkl', 'rb') as f:
    train_set = pickle.load(f)
    valid_set = pickle.load(f)
    test_set = pickle.load(f)

with open('/content/list_epinions.pkl', 'rb') as f:
    u_items_list = pickle.load(f)
    u_users_list = pickle.load(f)
    u_users_items_list = pickle.load(f)
    i_users_list = pickle.load(f)
    (user_count, item_count, rate_count) = pickle.load(f)

In [36]:
train_data = GraphDataset(train_set, u_items_list, u_users_list, u_users_items_list, i_users_list)
valid_data = GraphDataset(valid_set, u_items_list, u_users_list, u_users_items_list, i_users_list)
test_data = GraphDataset(test_set, u_items_list, u_users_list, u_users_items_list, i_users_list)

In [37]:
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True, collate_fn = collate_fn)
valid_loader = DataLoader(valid_data, batch_size = batch_size, shuffle = False, collate_fn = collate_fn)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False, collate_fn = collate_fn)

In [38]:
len(train_loader)

5704

### Create the model and set up training process

In [39]:
model = GraphRec(user_count+1, item_count+1, rate_count+1, embed_dim).to(device)

In [40]:
optimizer = torch.optim.RMSprop(model.parameters(), learning_rate)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 4, gamma = 0.1)

### Train the model

In [46]:
for epoch in range(n_epochs):

    # Training step
    model.train()
    s_loss = 0
    for i, (uids, iids, labels, u_items, u_users, u_users_items, i_users) in tqdm(enumerate(train_loader), total=len(train_loader)):
        uids = uids.to(device)
        iids = iids.to(device)
        labels = labels.to(device)
        u_items = u_items.to(device)
        u_users = u_users.to(device)
        u_users_items = u_users_items.to(device)
        i_users = i_users.to(device)
        
        optimizer.zero_grad()
        outputs = model(uids, iids, u_items, u_users, u_users_items, i_users)
        loss = criterion(outputs, labels.unsqueeze(1))

        loss.backward()
        optimizer.step()

        loss_val = loss.item()
        s_loss += loss_val
        
        iter_num = epoch * len(train_loader) + i + 1

    # Validate step
    model.eval()
    errors = []
    with torch.no_grad():
        for uids, iids, labels, u_items, u_users, u_users_items, i_users in tqdm(valid_loader):
            uids = uids.to(device)
            iids = iids.to(device)
            labels = labels.to(device)
            u_items = u_items.to(device)
            u_users = u_users.to(device)
            u_users_items = u_users_items.to(device)
            i_users = i_users.to(device)
            preds = model(uids, iids, u_items, u_users, u_users_items, i_users)
            error = torch.abs(preds.squeeze(1) - labels)
            errors.extend(error.data.cpu().numpy().tolist())
    
    mae = np.mean(errors)
    rmse = np.sqrt(np.mean(np.power(errors, 2)))

    scheduler.step()

    ckpt_dict = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }

    torch.save(ckpt_dict, 'trained_models/latest_checkpoint.pth')

    if epoch == 0:
        best_mae = mae
    elif mae < best_mae:
        best_mae = mae
        torch.save(ckpt_dict, 'trained_models/best_checkpoint_{}.pth'.format(embed_dim))

    print('Epoch {} validation: MAE: {:.4f}, RMSE: {:.4f}, Best MAE: {:.4f}'.format(epoch+1, mae, rmse, best_mae))

100%|██████████| 713/713 [00:42<00:00, 16.87it/s]


Epoch 1 validation: MAE: 1.2495, RMSE: 3.4318, Best MAE: 1.2495


100%|██████████| 713/713 [00:42<00:00, 16.85it/s]


Epoch 2 validation: MAE: 1.2535, RMSE: 3.4167, Best MAE: 1.2495


100%|██████████| 713/713 [00:42<00:00, 16.96it/s]


...

Epoch 8 validation: MAE: 1.2354, RMSE: 3.3996, Best MAE: 1.2354


100%|██████████| 713/713 [00:42<00:00, 16.93it/s]


Epoch 9 validation: MAE: 1.2349, RMSE: 3.4000, Best MAE: 1.2349


100%|██████████| 713/713 [00:42<00:00, 16.90it/s]


Epoch 10 validation: MAE: 1.2350, RMSE: 3.4004, Best MAE: 1.2349


## 6&nbsp;&nbsp;Testing <a class="anchor" id="testing"></a> [🔝](#contents)

In [48]:
embed_dim = 64
checkpoint = torch.load('trained_models/best_checkpoint_{}.pth'.format(embed_dim))
model = GraphRec(user_count+1, item_count+1, rate_count+1, embed_dim).to(device)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [49]:
model.eval()
test_errors = []
with torch.no_grad():
    for uids, iids, labels, u_items, u_users, u_users_items, i_users in tqdm(test_loader):
        uids = uids.to(device)
        iids = iids.to(device)
        labels = labels.to(device)
        u_items = u_items.to(device)
        u_users = u_users.to(device)
        u_users_items = u_users_items.to(device)
        i_users = i_users.to(device)
        preds = model(uids, iids, u_items, u_users, u_users_items, i_users)
        error = torch.abs(preds.squeeze(1) - labels)
        test_errors.extend(error.data.cpu().numpy().tolist())

test_mae = np.mean(test_errors)
test_rmse = np.sqrt(np.mean(np.power(test_errors, 2)))
print('Test: MAE: {:.4f}, RMSE: {:.4f}'.format(test_mae, test_rmse))

100%|██████████| 713/713 [00:43<00:00, 16.35it/s]

Test: MAE: 1.2399, RMSE: 3.4053


## 7&nbsp;&nbsp;Results <a class="anchor" id="results"></a> [🔝](#contents)

Tables 

## 8&nbsp;&nbsp;References <a class="anchor" id="references"></a> [🔝](#contents)

<sup>1</sup>Antonia Creswell, Tom White, Vincent Dumoulin, Kai Arulkumaran, Biswa Sengupta, and Anil A Bharath. “Generative Adversarial Networks: An Overview”. In: *IEEE Signal Processing Magazine* 35.1 (2018), pp. 53–65.<br/>


**To cite, please use the following information:**

_BibTeX_:

`@misc{asif_nanjangud_2022,`<br />
    `title = {Recommendation System using Graph Neural Networks},`<br />
    `url = {https://github.com/mughees-asif/__________},`<br />
    `journal = {GitHub},`<br />
    `publisher = {Mughees Asif},`<br />
    `author = {Asif, Mughees and Nanjangud, Angadh},`<br />
    `year = {2022},`<br />
    `month = {Aug}`<br />
   `}`

_Harvard_:

`Asif, M., and Nanjangud, A.. (2022). Recommendation System using Graph Neural Networks.`

_APA_:

`Asif, M., & Nanjangud, A.. (2022). Recommendation System using Graph Neural Networks.`

<h1 style='color:purple'><center><a href='#contents'>🔝</a>&#8592;&#8592;&#8592;&#8592; END &#8594;&#8594;&#8594;&#8594;<a href='#contents'>🔝</a></center></h1>